In [20]:
import rasterio as rio
from rasterio.plot import plotting_extent
import rasterio.plot
import numpy as np
#from topolib.subsetDat import subsetBBox;
#from topolib import gda_lib;
from pyproj import Proj, transform
from icepyx import icesat2data as ipd
import matplotlib.pyplot as plt 
import sys
from pathlib import Path
import cartopy.crs as ccrs
import cartopy
from astropy.time import Time
import h5py
import readers as rd

#%matplotlib inline
%matplotlib widget

In [18]:
!pwd

/home/jovyan/SeasonalSnow/contributors/IsobelLawrence


# Find ICESat2 data within domain near Yellowknife-Henderson weather station
Yellowknife-Henderson weather station is one of the few in the Northwest Territories than records daily snowfall

See: https://climate.weather.gc.ca/climate_data/daily_data_e.html?hlyRange=%7C&dlyRange=2006-07-01%7C2020-05-31&mlyRange=2006-07-01%7C2007-11-01&StationID=45467&Prov=NT&urlExtension=_e.html&searchType=stnProv&optLimit=yearRange&StartYear=2019&EndYear=2020&selRowPerPage=25&Line=71&Month=12&Day=16&lstProvince=NT&timeframe=2&Year=2019

Weather station is at lat/lon: 62°27'00.000"N, 114°23'00.000"W, next to 'Frame Lake'. Let's create bounding box around Slave lake and then search for IS2 ATL03 data

In [3]:
#'LowerLeft_Lon': L,
#'LowerLeft_Lat': B,
#'UpperRight_Lon': R,
#'UpperRight_Lat': T

#cmd+/ to comment out a block of code!

L=-114.3618
B=62.4145
R=-114.3025
T=62.5077

# L=-115
# B=62
# R=-113
# T=63

#whole of Great Slave Lake:
L,B,R,T=-118, 60.5, -110, 63

L,B,R,T=-115.5, 60.8, -113.5, 62.6

Plot this region

In [43]:
#var= 'heights' #choose which variable we want to plot

## we will want to set colorbar parameters based on the chosen variable
#vmin=-10
#vmax=30
#ticks=np.arange(-20,100,5)


plt.figure(figsize=(8,8), dpi= 90)
ax = plt.axes(projection=ccrs.NorthPolarStereo(central_longitude=-114)) # choose polar sterographic for projection
ax.coastlines(resolution='50m', color='black', linewidth=1)
ax.add_feature(cartopy.feature.LAKES, alpha=0.5)
ax.set_extent([-160, 0, 50, 60], ccrs.PlateCarree())
#ax.gridlines(draw_labels=True, xlocs=[150, 152, 154, 155])
plt.scatter(L,B,s=10,color='red',transform=ccrs.PlateCarree())
plt.scatter(L,T,s=10,color='red',transform=ccrs.PlateCarree())
plt.scatter(R,T,s=10,color='red',transform=ccrs.PlateCarree())
plt.scatter(R,B,s=10,color='red',transform=ccrs.PlateCarree())
#plt.scatter(df03['lons'][::1000], df03['lats'][::1000],c=df03[var][::1000], cmap='plasma', vmin=vmin,vmax=vmax,transform=ccrs.PlateCarree())
#plt.colorbar(label=var, shrink=0.5, ticks=ticks,extend='both')

#rect1 = patches.Rectangle((x1, y1), xwidth, ywidth,linewidth = 1,
#                              edgecolor = 0,  facecolor='none')
#ax = plt.gca()
#    ax.add_patch(rect1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:
# Our data folder 
data_home = Path('/home/jovyan/shared/data-Isobel_Rosie/ATL03/Whole_SL_Lake')
                 
# Create folder if it doesn't exist
data_home.mkdir(exist_ok=True)

In [6]:
!ls -l /home/jovyan/shared/data-Isobel_Rosie/ATL03/Whole_SL_Lake/

total 0


In [7]:
short_name = 'ATL03'
spatial_extent = [L,B,R,T]
date_range = ['2019-07-01','2020-06-30']
#date_range = ['2019-11-26','2019-11-26']
version='003'

# spatial_extent = [148, -81, 162, -80]  # Byrd
# date_range = ['2018-10-14','2018-12-22']

region = ipd.Icesat2Data(short_name, spatial_extent, date_range, version=version)

In [8]:
print('product:    ', region.dataset)
print('dates:      ', region.dates)
print('start time: ', region.start_time)
print('end time:   ', region.end_time)
print('version:    ', region.dataset_version)
print('extent:     ', region.spatial_extent)

print('\nDATA:')
print('\n'.join([str(item) for item in region.avail_granules().items()]))

region.visualize_spatial_extent()

product:     ATL03
dates:       ['2019-07-01', '2020-06-30']
start time:  00:00:00
end time:    23:59:59
version:     003
extent:      ['bounding box', [-115.5, 60.8, -113.5, 62.6]]

DATA:
('Number of available granules', 69)
('Average size of granules (MB)', 1320.6068256074084)
('Total size of all granules (MB)', 91121.8709669112)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
#build and view the parameters that will be submitted in our query
region.CMRparams

{'short_name': 'ATL03',
 'version': '003',
 'temporal': '2019-07-01T00:00:00Z,2020-06-30T23:59:59Z',
 'bounding_box': '-115.5,60.8,-113.5,62.6'}

In [302]:
#get a list of the available granule IDs that meet your search criteria
region.avail_granules(ids=True) 

['ATL03_20190729091103_04780403_003_01.h5',
 'ATL03_20190730213017_05010405_003_01.h5',
 'ATL03_20190802090244_05390403_003_01.h5',
 'ATL03_20190803212159_05620405_003_01.h5',
 'ATL03_20190806085425_06000403_003_01.h5',
 'ATL03_20190807211338_06230405_003_01.h5',
 'ATL03_20190823075529_08590403_003_01.h5',
 'ATL03_20190827074710_09200403_003_01.h5',
 'ATL03_20190828200623_09430405_003_01.h5',
 'ATL03_20190831073850_09810403_003_01.h5',
 'ATL03_20190901195804_10040405_003_01.h5',
 'ATL03_20190904073030_10420403_003_01.h5',
 'ATL03_20190905194943_10650405_003_01.h5',
 'ATL03_20190909194125_11260405_003_01.h5',
 'ATL03_20190921063134_13010403_003_01.h5',
 'ATL03_20190925062316_13620403_003_01.h5',
 'ATL03_20190926184230_13850405_003_01.h5',
 'ATL03_20190929061456_00360503_003_01.h5',
 'ATL03_20190930183409_00590505_003_01.h5',
 'ATL03_20191003060637_00970503_003_01.h5',
 'ATL03_20191004182551_01200505_003_01.h5',
 'ATL03_20191008181730_01810505_003_01.h5',
 'ATL03_20191024045920_04170503_

In [20]:
region.granules.wanted()

AttributeError: 'Granules' object has no attribute 'wanted'

In [42]:
#polygons = [granule['polygons'] for granule in region.granules.avail]

In [153]:
#polygons[0]

# Download the data

In [10]:
name = 'isobelrlawrence'
email = 'isobel.lawrence.15@ucl.ac.uk'
region.earthdata_login(name, email)
region.download_granules(data_home)

Earthdata Login password:  ··········


Total number of data order requests is  7  for  69  granules.
Data request  1  of  7  is submitting to NSIDC
order ID:  5000000702413
Initial status of your order request at NSIDC is:  processing
Your order status is still  processing  at NSIDC. Please continue waiting... this may take a few moments.
Your order status is still  processing  at NSIDC. Please continue waiting... this may take a few moments.
Your order status is still  processing  at NSIDC. Please continue waiting... this may take a few moments.
Your order status is still  processing  at NSIDC. Please continue waiting... this may take a few moments.
Your order status is still  processing  at NSIDC. Please continue waiting... this may take a few moments.
Your order status is still  processing  at NSIDC. Please continue waiting... this may take a few moments.
Your order is: complete
Data request  2  of  7  is submitting to NSIDC
order ID:  5000000702416
Initial status of your order request at NSIDC is:  processing
Your order

KeyboardInterrupt: 

In [154]:
name = 'isobelrlawrence'
email = 'isobel.lawrence.15@ucl.ac.uk'

# for file in region.avail_granules(ids=True):
#     print(file)

if not list(data_home.glob('*.h5*')):
    print('you dont have the files')
    region.earthdata_login(name, email)
    region.download_granules(data_home)


you dont have the files


Earthdata Login password:  ··········


['Invalid username or password, please retry.']


Please re-enter your Earthdata user ID:  isobelrlawrence
Earthdata Login password:  ··········


Total number of data order requests is  3  for  30  granules.
Data request  1  of  3  is submitting to NSIDC
order ID:  5000000702269
Initial status of your order request at NSIDC is:  processing
Your order status is still  processing  at NSIDC. Please continue waiting... this may take a few moments.
Your order status is still  processing  at NSIDC. Please continue waiting... this may take a few moments.
Your order status is still  processing  at NSIDC. Please continue waiting... this may take a few moments.
Your order is: complete
Data request  2  of  3  is submitting to NSIDC
order ID:  5000000702270
Initial status of your order request at NSIDC is:  processing
Your order status is still  processing  at NSIDC. Please continue waiting... this may take a few moments.
Your order status is still  processing  at NSIDC. Please continue waiting... this may take a few moments.
Your order status is still  processing  at NSIDC. Please continue waiting... this may take a few moments.
Your order

In [157]:
files = list(data_home.glob('*003_01.h5'))

for f in files: print(f)
print('Total number of files:', len(files))

/home/jovyan/shared/data-Isobel_Rosie/ATL03/SL_Lake/processed_ATL03_20190327150344_13620203_003_01.h5
/home/jovyan/shared/data-Isobel_Rosie/ATL03/SL_Lake/processed_ATL03_20190505014217_05620305_003_01.h5
/home/jovyan/shared/data-Isobel_Rosie/ATL03/SL_Lake/processed_ATL03_20190304043840_10040205_003_01.h5
/home/jovyan/shared/data-Isobel_Rosie/ATL03/SL_Lake/processed_ATL03_20191225020303_13620503_003_01.h5
/home/jovyan/shared/data-Isobel_Rosie/ATL03/SL_Lake/processed_ATL03_20190827074710_09200403_003_01.h5
/home/jovyan/shared/data-Isobel_Rosie/ATL03/SL_Lake/processed_ATL03_20200201124138_05620605_003_01.h5
/home/jovyan/shared/data-Isobel_Rosie/ATL03/SL_Lake/processed_ATL03_20190429133124_04780303_003_01.h5
/home/jovyan/shared/data-Isobel_Rosie/ATL03/SL_Lake/processed_ATL03_20200403094520_01200705_003_01.h5
/home/jovyan/shared/data-Isobel_Rosie/ATL03/SL_Lake/processed_ATL03_20191126032701_09200503_003_01.h5
/home/jovyan/shared/data-Isobel_Rosie/ATL03/SL_Lake/processed_ATL03_20200127003045

# Plot the ATL03 data 
From sea-ice ATL03 Notebook

In [13]:
%%bash

rm contain_data.txt
for file in /home/jovyan/shared/data-Isobel_Rosie/ATL03/Whole_SL_Lake/*003_01.h5
#do echo ${file}
do
if h5ls -r ${file} | grep '/heights/'
then
echo ${file} >> contain_data.txt
else
rm ${file}
fi
done

/gt1l/heights/delta_time Dataset {282433/Inf}
/gt1l/heights/dist_ph_across Dataset {282433/Inf}
/gt1l/heights/dist_ph_along Dataset {282433/Inf}
/gt1l/heights/h_ph       Dataset {282433/Inf}
/gt1l/heights/lat_ph     Dataset {282433/Inf}
/gt1l/heights/lon_ph     Dataset {282433/Inf}
/gt1l/heights/pce_mframe_cnt Dataset {282433/Inf}
/gt1l/heights/ph_id_channel Dataset {282433/Inf}
/gt1l/heights/ph_id_count Dataset {282433/Inf}
/gt1l/heights/ph_id_pulse Dataset {282433/Inf}
/gt1l/heights/signal_conf_ph Dataset {282433/Inf, 5}
/gt1r/heights/delta_time Dataset {65804/Inf}
/gt1r/heights/dist_ph_across Dataset {65804/Inf}
/gt1r/heights/dist_ph_along Dataset {65804/Inf}
/gt1r/heights/h_ph       Dataset {65804/Inf}
/gt1r/heights/lat_ph     Dataset {65804/Inf}
/gt1r/heights/lon_ph     Dataset {65804/Inf}
/gt1r/heights/pce_mframe_cnt Dataset {65804/Inf}
/gt1r/heights/ph_id_channel Dataset {65804/Inf}
/gt1r/heights/ph_id_count Dataset {65804/Inf}
/gt1r/heights/ph_id_pulse Dataset {65804/Inf}
/gt1r

In [14]:
!ls /home/jovyan/shared/data-Isobel_Rosie/ATL03/Whole_SL_Lake/*003_01.h5 | wc -l

32


In [15]:
%%bash

for file in /home/jovyan/shared/data-Isobel_Rosie/ATL03/Whole_SL_Lake/*003_01.h5
do
echo $file
h5ls -r ${file} | grep '/heights/h_ph'
echo
done

/home/jovyan/shared/data-Isobel_Rosie/ATL03/Whole_SL_Lake/processed_ATL03_20190729091103_04780403_003_01.h5
/gt1l/heights/h_ph       Dataset {282433/Inf}
/gt1r/heights/h_ph       Dataset {65804/Inf}
/gt2l/heights/h_ph       Dataset {218619/Inf}
/gt2r/heights/h_ph       Dataset {77533/Inf}
/gt3l/heights/h_ph       Dataset {281004/Inf}
/gt3r/heights/h_ph       Dataset {68734/Inf}

/home/jovyan/shared/data-Isobel_Rosie/ATL03/Whole_SL_Lake/processed_ATL03_20190730213017_05010405_003_01.h5
/gt1l/heights/h_ph       Dataset {993979/Inf}
/gt1r/heights/h_ph       Dataset {236615/Inf}
/gt2l/heights/h_ph       Dataset {841023/Inf}
/gt2r/heights/h_ph       Dataset {298182/Inf}
/gt3l/heights/h_ph       Dataset {999994/Inf}
/gt3r/heights/h_ph       Dataset {236856/Inf}

/home/jovyan/shared/data-Isobel_Rosie/ATL03/Whole_SL_Lake/processed_ATL03_20190802090244_05390403_003_01.h5
/gt1l/heights/h_ph       Dataset {5129/Inf}
/gt1r/heights/h_ph       Dataset {815/Inf}
/gt2l/heights/h_ph       Dataset {3883

# 1=foreward orientation: right beams are strong, left are weak
# 0=backward orientation: left beams are strong, right are weak

In [33]:
from astropy.time import Time
import warnings
warnings.filterwarnings('ignore')
%matplotlib widget

In [291]:
L,B,R,T=-115.5, 60.8, -113.5, 62.6

In [54]:
files=np.genfromtxt('contain_data.txt',dtype='str')
#beams=['gt1','gt2','gt2','gt2','gt2','gt3']

# files=['/home/jovyan/shared/data-Isobel_Rosie/ATL03/Whole_SL_Lake/processed_ATL03_20190827074710_09200403_003_01.h5',\
#        '/home/jovyan/shared/data-Isobel_Rosie/ATL03/Whole_SL_Lake/processed_ATL03_20190831073850_09810403_003_01.h5',\
#        '/home/jovyan/shared/data-Isobel_Rosie/ATL03/Whole_SL_Lake/processed_ATL03_20191102170152_05620505_003_01.h5',\
#        '/home/jovyan/shared/data-Isobel_Rosie/ATL03/Whole_SL_Lake/processed_ATL03_20191130031840_09810503_003_01.h5']

files=['/home/jovyan/shared/data-Isobel_Rosie/ATL03/Whole_SL_Lake/processed_ATL03_20190831073850_09810403_003_01.h5',\
       '/home/jovyan/shared/data-Isobel_Rosie/ATL03/Whole_SL_Lake/processed_ATL03_20191130031840_09810503_003_01.h5']

ind=-1
for file in files:

    ind=ind+1
    print(file)
    
    f = h5py.File(file, 'r')
    orient=f['orbit_info/sc_orient'][0]
    
    if orient==1: 
        # foreward
        #beam=beams[ind]+'r'
        beam='gt3'+'r'
    elif orient==0:  
        # backward
        #beam=beams[ind]+'l'
        beam='gt3'+'l'
    #print(beam)
        
    #beam='gt2r'
    df03=rd.getATL03(f,beam)
    #print(df03.head())
    
    
    # Plot a map:
    
    var= 'heights' #choose which variable we want to plot

    ## we will want to set colorbar parameters based on the chosen variable
    #vmin=-10
    #vmax=30
    #ticks=np.arange(-20,100,5)


    plt.figure(figsize=(8,8), dpi= 90)
    ax = plt.axes(projection=ccrs.NorthPolarStereo(central_longitude=-114)) # choose polar sterographic for projection
    ax.coastlines(resolution='50m', color='black', linewidth=1)
    ax.add_feature(cartopy.feature.LAKES,edgecolor='blue',facecolor='None')
    ax.set_extent([L,R,B,T], ccrs.PlateCarree())
    plt.scatter(df03['lons'][::1000], df03['lats'][::1000],c=df03[var][::1000], cmap='plasma', transform=ccrs.PlateCarree())
    plt.colorbar(label=var, shrink=0.5,extend='both')
    plt.show()
    
   # stop
    
    ## create bounding box of interest:
    lonmin,lonmax,latmin,latmax= [L,B,R,T]

    bbox=(df03['lons']>lonmin)&(df03['lons']<lonmax)&(df03['lats']>latmin)&(df03['lats']<latmax)
    df_cut=df03[bbox]

    ## converting GPS time to UTC time
    epoch=f['/ancillary_data/atlas_sdp_gps_epoch'][0]
    df_cut['time']=Time(epoch+df_cut['dt'],format='gps').utc.datetime

    ## calculate along track distance relative to the beginning of the cut segment
    df_cut['AT_dist']=df_cut.x-df_cut.x.values[0]

    #print(df_cut.head())
    
    plt.subplots(1,1,figsize=(10,2))
    plt.scatter(df_cut.AT_dist.values[df_cut.conf<3],df_cut.heights.values[df_cut.conf<3],c='r',s=1)
    plt.scatter(df_cut.AT_dist.values[df_cut.conf>2],df_cut.heights.values[df_cut.conf>2],c='g',s=1)
    plt.xlabel('alongtrack distance (m)',fontsize=20)
    plt.ylabel('heights',fontsize=20)
    plt.ylim(50,250);
    #plt.ticklabel_format(style="plain",axis="x", useOffset=False)
    plt.xlim(df_cut.AT_dist.values[0],df_cut.AT_dist.values[-1])
    plt.show()

/home/jovyan/shared/data-Isobel_Rosie/ATL03/Whole_SL_Lake/processed_ATL03_20190831073850_09810403_003_01.h5


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/jovyan/shared/data-Isobel_Rosie/ATL03/Whole_SL_Lake/processed_ATL03_20191130031840_09810503_003_01.h5


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [58]:
import pandas as pd
import datetime
import matplotlib.gridspec as gridspec
import numpy as np

weather19 = pd.read_csv('en_climate_daily_NT_2204110_2019_P1D.csv')

weather19.columns = [c.strip().lower().replace(' ', '_').replace('/', '_').replace('(','_').replace(')','_').replace('°','') for c in weather19.columns]

weather20 = pd.read_csv('en_climate_daily_NT_2204110_2020_P1D.csv')

weather20.columns = [c.strip().lower().replace(' ', '_').replace('/', '_').replace('(','_').replace(')','_').replace('°','') for c in weather20.columns]



#print(weather.columns)

w_dt19 = [datetime.datetime.strptime(d, '%Y-%m-%d') for d in weather19.date_time]
w_dt20 = [datetime.datetime.strptime(d, '%Y-%m-%d') for d in weather20.date_time]

w_dt = w_dt19.append(w_dt20)

# print('lenw',len(w_dt))


# print(w_dt)

w_tot_snow = weather19.total_snow__cm_
w_tot_snow.append(weather20.total_snow__cm_)

# help(w_tot_snow)

# date_str = '29-12-2017' # The date - 29 Dec 2017
# format_str = '%d/%m/%Y' # The format
# datetime_obj = datetime.datetime.strptime(date_str, format_str)
# print(datetime_obj.date())

figsize = (12,9)
fig = plt.figure(figsize=figsize)
# gs = gridspec.GridSpec(2, 1, figure=fig)

# print('len1',len(weather19.date_time),len(weather19.total_snow__cm_),len(weather20.date_time),len(weather20.total_snow__cm_))

plt.plot(weather19.date_time, weather19.total_snow__cm_,color='r')
plt.plot(weather20.date_time, weather20.total_snow__cm_,color='r')

plt.xlabel('date')
plt.ylabel('snowfall (cm)')
#locs, labels = xticks()
plt.xticks(rotation=90,fontsize=2,color='white')

plt.show()

i = np.where(weather19.total_snow__cm_ > 0)[0]
for i in i:
    plt.text(weather19.date_time[i],weather19.total_snow__cm_[i],weather19.date_time[i],rotation = 90,fontsize = 8)
    
i = np.where(weather20.total_snow__cm_ > 0)[0]
for i in i:
    plt.text(weather20.date_time[i],weather20.total_snow__cm_[i],weather20.date_time[i],rotation = 90,fontsize = 8)

# i = np.where(weather20.date_time == '2019-08-27')  

print(len(weather19.date_time))

for i in (242,333):
    print(i)
    print(weather19.date_time[i])
    plt.text(weather19.date_time[i],6,'IS2 '+ weather19.date_time[i],rotation = 90,fontsize = 8,color = 'b')
    plt.plot((weather19.date_time[i],weather19.date_time[i]),(0.1,5.9),color = 'b')
    

# i = 239
# print(i)
# print(weather19.date_time[i])
# plt.text(weather19.date_time[i],6,'IS2',rotation = 90,fontsize = 8,color='b')

# i = 306
# i = print(i)
# print(weather19.date_time[i])
# plt.text(weather19.date_time[i],6,'IS2',rotation = 90,fontsize = 8,color='b')

# i = 331
# i = print(i)
# print(weather19.date_time[i])
# plt.text(weather19.date_time[i],6,'IS2',rotation = 90,fontsize = 8,color='b')

# figsize = (12,9)
# fig = plt.figure(figsize=figsize)
# gs = gridspec.GridSpec(2, 1, figure=fig)


# # print('len',len(w_dt),len(w_tot_snow))

# # plt.plot(w_dt, w_tot_snow)
# plt.xlabel('date')
# plt.ylabel('snowfall (cm)')
# #locs, labels = xticks()
# plt.xticks(rotation=90,fontsize=6)

# plt.show()

# 20190729, 20190827, 20190831, 20191003, 20191102, 20191130

# 20190827 134
# 20190831 135
# 20191102 130
# 20191127 130

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

365
242
2019-08-31
333
2019-11-30


In [49]:
files=np.genfromtxt('contain_data.txt',dtype='str')
#beams=['gt1','gt2','gt2','gt2','gt2','gt3']

file='/home/jovyan/shared/data-Isobel_Rosie/ATL03/Whole_SL_Lake/processed_ATL03_20190831073850_09810403_003_01.h5'

#ind=-1
#for file in files:

#ind=ind+1
print(file)

f = h5py.File(file, 'r')
orient=f['orbit_info/sc_orient'][0]

if orient==1: 
    # foreward
    #beam=beams[ind]+'r'
    beam='gt3'+'r'
elif orient==0:  
    # backward
    #beam=beams[ind]+'l'
    beam='gt3'+'l'
#print(beam)

#beam='gt2r'
df03=rd.getATL03(f,beam)
#print(df03.head())



file='/home/jovyan/shared/data-Isobel_Rosie/ATL03/Whole_SL_Lake/processed_ATL03_20191130031840_09810503_003_01.h5'

#ind=-1
#for file in files:

#ind=ind+1
print(file)

f = h5py.File(file, 'r')
orient=f['orbit_info/sc_orient'][0]

if orient==1: 
    # foreward
    #beam=beams[ind]+'r'
    beam='gt3'+'r'
elif orient==0:  
    # backward
    #beam=beams[ind]+'l'
    beam='gt3'+'l'
#print(beam)

#beam='gt2r'
df03_2=rd.getATL03(f,beam)
#print(df03.head())


# Plot a map:

var= 'heights' #choose which variable we want to plot

## we will want to set colorbar parameters based on the chosen variable
#vmin=-10
#vmax=30
#ticks=np.arange(-20,100,5)


plt.figure(figsize=(8,8), dpi= 90)
ax = plt.axes(projection=ccrs.NorthPolarStereo(central_longitude=-114)) # choose polar sterographic for projection
ax.coastlines(resolution='50m', color='black', linewidth=1)
ax.add_feature(cartopy.feature.LAKES,edgecolor='blue',facecolor='None')
ax.set_extent([L,R,B,T], ccrs.PlateCarree())
plt.scatter(df03['lons'][::1000], df03['lats'][::1000],c=df03[var][::1000], cmap='plasma', transform=ccrs.PlateCarree())

plt.scatter(df03_2['lons'][::1000], df03_2['lats'][::1000],c=df03_2[var][::1000], cmap='plasma', transform=ccrs.PlateCarree())

plt.colorbar(label=var, shrink=0.5,extend='both')
plt.show()

# stop






#file='/home/jovyan/shared/data-Isobel_Rosie/ATL03/Whole_SL_Lake/processed_ATL03_20191102170152_05620505_003_01.h5'
file='/home/jovyan/shared/data-Isobel_Rosie/ATL03/Whole_SL_Lake/processed_ATL03_20190831073850_09810403_003_01.h5'
print(file)

f = h5py.File(file, 'r')
orient=f['orbit_info/sc_orient'][0]

if orient==1: 
    # foreward
    #beam=beams[ind]+'r'
    beam='gt3'+'r'
elif orient==0:  
    # backward
    #beam=beams[ind]+'l'
    beam='gt3'+'l'
#print(beam)

#beam='gt2r'
df03=rd.getATL03(f,beam)
#print(df03.head())


## create bounding box of interest:
lonmin,lonmax,latmin,latmax= [L,B,R,T]

bbox=(df03['lons']>lonmin)&(df03['lons']<lonmax)&(df03['lats']>latmin)&(df03['lats']<latmax)
df_cut=df03[bbox]

## converting GPS time to UTC time
epoch=f['/ancillary_data/atlas_sdp_gps_epoch'][0]
df_cut['time']=Time(epoch+df_cut['dt'],format='gps').utc.datetime

## calculate along track distance relative to the beginning of the cut segment
df_cut['AT_dist']=df_cut.x-df_cut.x.values[0]

#print(df_cut.head())

plt.subplots(1,1,figsize=(10,2))
plt.scatter(df_cut.AT_dist.values[df_cut.conf<3],df_cut.heights.values[df_cut.conf<3],c='pink',s=1)
plt.scatter(df_cut.AT_dist.values[df_cut.conf>2],df_cut.heights.values[df_cut.conf>2],c='red',s=1)
plt.xlabel('alongtrack distance (m)',fontsize=20)
plt.ylabel('heights',fontsize=20)
#plt.ylim(125,135);
#plt.ticklabel_format(style="plain",axis="x", useOffset=False)
#plt.xlim(df_cut.AT_dist.values[0],df_cut.AT_dist.values[-1])





file='/home/jovyan/shared/data-Isobel_Rosie/ATL03/Whole_SL_Lake/processed_ATL03_20191130031840_09810503_003_01.h5'
print(file)

f = h5py.File(file, 'r')
orient=f['orbit_info/sc_orient'][0]

if orient==1: 
    # foreward
    #beam=beams[ind]+'r'
    beam='gt3'+'r'
elif orient==0:  
    # backward
    #beam=beams[ind]+'l'
    beam='gt3'+'l'
#print(beam)

#beam='gt2r'
df03=rd.getATL03(f,beam)
#print(df03.head())


## create bounding box of interest:
lonmin,lonmax,latmin,latmax= [L,B,R,T]

bbox=(df03['lons']>lonmin)&(df03['lons']<lonmax)&(df03['lats']>latmin)&(df03['lats']<latmax)
df_cut=df03[bbox]

## converting GPS time to UTC time
epoch=f['/ancillary_data/atlas_sdp_gps_epoch'][0]
df_cut['time']=Time(epoch+df_cut['dt'],format='gps').utc.datetime

## calculate along track distance relative to the beginning of the cut segment
df_cut['AT_dist']=df_cut.x-df_cut.x.values[0]

#print(df_cut.head())

#plt.subplots(1,1,figsize=(10,2))
plt.scatter(df_cut.AT_dist.values[df_cut.conf<3],df_cut.heights.values[df_cut.conf<3],c='lightblue',s=1)
plt.scatter(df_cut.AT_dist.values[df_cut.conf>2],df_cut.heights.values[df_cut.conf>2],c='blue',s=1)
#plt.xlabel('alongtrack distance (m)',fontsize=20)
#plt.ylabel('heights',fontsize=20)
plt.ylim(111,227);
#plt.ticklabel_format(style="plain",axis="x", useOffset=False)
#plt.xlim(df_cut.AT_dist.values[0],df_cut.AT_dist.values[-1])
plt.show()











#file='/home/jovyan/shared/data-Isobel_Rosie/ATL03/Whole_SL_Lake/processed_ATL03_20191102170152_05620505_003_01.h5'
file='/home/jovyan/shared/data-Isobel_Rosie/ATL03/Whole_SL_Lake/processed_ATL03_20190831073850_09810403_003_01.h5'
print(file)

f = h5py.File(file, 'r')
orient=f['orbit_info/sc_orient'][0]

if orient==1: 
    # foreward
    #beam=beams[ind]+'r'
    beam='gt3'+'r'
elif orient==0:  
    # backward
    #beam=beams[ind]+'l'
    beam='gt3'+'l'
#print(beam)

#beam='gt2r'
df03=rd.getATL03(f,beam)
#print(df03.head())


## create bounding box of interest:
lonmin,lonmax,latmin,latmax= [L,B,R,T]

bbox=(df03['lons']>lonmin)&(df03['lons']<lonmax)&(df03['lats']>latmin)&(df03['lats']<latmax)
df_cut=df03[bbox]

## converting GPS time to UTC time
epoch=f['/ancillary_data/atlas_sdp_gps_epoch'][0]
df_cut['time']=Time(epoch+df_cut['dt'],format='gps').utc.datetime

## calculate along track distance relative to the beginning of the cut segment
df_cut['AT_dist']=df_cut.x-df_cut.x.values[0]

#print(df_cut.head())

plt.subplots(1,1,figsize=(10,2))
plt.scatter(df_cut.AT_dist.values[df_cut.conf<3],df_cut.heights.values[df_cut.conf<3],c='pink',s=1)
plt.scatter(df_cut.AT_dist.values[df_cut.conf>2],df_cut.heights.values[df_cut.conf>2],c='red',s=1)
plt.xlabel('alongtrack distance (m)',fontsize=20)
plt.ylabel('heights',fontsize=20)
#plt.ylim(125,135);
#plt.ticklabel_format(style="plain",axis="x", useOffset=False)
#plt.xlim(df_cut.AT_dist.values[0],df_cut.AT_dist.values[-1])


file='/home/jovyan/shared/data-Isobel_Rosie/ATL03/Whole_SL_Lake/processed_ATL03_20191130031840_09810503_003_01.h5'
print(file)

f = h5py.File(file, 'r')
orient=f['orbit_info/sc_orient'][0]

if orient==1: 
    # foreward
    #beam=beams[ind]+'r'
    beam='gt3'+'r'
elif orient==0:  
    # backward
    #beam=beams[ind]+'l'
    beam='gt3'+'l'
#print(beam)

#beam='gt2r'
df03=rd.getATL03(f,beam)
#print(df03.head())


## create bounding box of interest:
lonmin,lonmax,latmin,latmax= [L,B,R,T]

bbox=(df03['lons']>lonmin)&(df03['lons']<lonmax)&(df03['lats']>latmin)&(df03['lats']<latmax)
df_cut=df03[bbox]

## converting GPS time to UTC time
epoch=f['/ancillary_data/atlas_sdp_gps_epoch'][0]
df_cut['time']=Time(epoch+df_cut['dt'],format='gps').utc.datetime

## calculate along track distance relative to the beginning of the cut segment
df_cut['AT_dist']=df_cut.x-df_cut.x.values[0]

#print(df_cut.head())

#plt.subplots(1,1,figsize=(10,2))
plt.scatter(df_cut.AT_dist.values[df_cut.conf<3],df_cut.heights.values[df_cut.conf<3],c='lightblue',s=1)
plt.scatter(df_cut.AT_dist.values[df_cut.conf>2],df_cut.heights.values[df_cut.conf>2],c='blue',s=1)
#plt.xlabel('alongtrack distance (m)',fontsize=20)
#plt.ylabel('heights',fontsize=20)
plt.ylim(125,137);
#plt.ticklabel_format(style="plain",axis="x", useOffset=False)
#plt.xlim(df_cut.AT_dist.values[0],df_cut.AT_dist.values[-1])

/home/jovyan/shared/data-Isobel_Rosie/ATL03/Whole_SL_Lake/processed_ATL03_20190831073850_09810403_003_01.h5
/home/jovyan/shared/data-Isobel_Rosie/ATL03/Whole_SL_Lake/processed_ATL03_20191130031840_09810503_003_01.h5


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/jovyan/shared/data-Isobel_Rosie/ATL03/Whole_SL_Lake/processed_ATL03_20190831073850_09810403_003_01.h5


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/jovyan/shared/data-Isobel_Rosie/ATL03/Whole_SL_Lake/processed_ATL03_20191130031840_09810503_003_01.h5
/home/jovyan/shared/data-Isobel_Rosie/ATL03/Whole_SL_Lake/processed_ATL03_20190831073850_09810403_003_01.h5


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/jovyan/shared/data-Isobel_Rosie/ATL03/Whole_SL_Lake/processed_ATL03_20191130031840_09810503_003_01.h5
